[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap12/cap12-main.ipynb)

# Capítulo 12 - Tarefas Continuadas: Formulação Alternativa


## Configurações Iniciais

In [1]:
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install gymnasium
    !pip install optuna

    # clone repository
    !git clone https://github.com/pablo-sampaio/rl_facil
    sys.path.append("/content/rl_facil")

    #clear_output()
else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )


In [2]:
import gymnasium as gym
import numpy as np
import optuna

from envs import TwoChoiceEnv, AccessControlEnv
from envs.wrappers import FromDiscreteTupleToDiscreteObs

from util.experiments import repeated_exec
from util.plot import plot_result, plot_multiple_results
from util.qtable_helper import epsilon_greedy

## 1 - Tarefa Continuada (Infinita)

Uma **tarefa continuada** (*continuing task*) é uma tarefa representada por um MDP sem estado terminal (que pode ser chamado de *MDP continuado* ou *MDP de horizonte infinito*).

Existem várias críticas quanto à aplicação de algoritmos baseados em *retornos descontados* nestes ambientes, conforme o artigo "*Discounted Reinforcement Learning is Not an Optimization Problem*" (Naik et al., 2019).

O artigo propõe o MDP abaixo para ilustrar as dificuldades de métodos como o *Q-Learning* e o *SARSA* em tarefas desse tipo:

<p align="center">
   <img src="https://github.com/pablo-sampaio/rl_facil/raw/main/cap12/two-choice-mdp-naik2019.jpg" alt="The Two-choice MDP (Naik et al., 2019)" width="500">
</p?>

Este MDP está implementado como ambiente `gym` na classe `util.env.TwoChoiceEnv`.

## 2 - Q-Learning (parando por passos)

Abaixo, segue o código do **Q-learning**, com uma pequena alteração em relação à implementação dada antes:
- o critério de parada agora é a *quantidade de passos*
- não importa a quantidade de episódios envolvida.

Este será o critério de parada na maioria dos algoritmos que veremos no futuro.

*Atenção: alguns gráficos vão mostrar o eixo "x" como "episódios", mas entenda como "passos".*

In [3]:
# Algoritmo Q-learning
def run_qlearning_step(env, total_steps, lr=0.1, gamma=0.95, epsilon=0.1, verbose=False):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n

    # inicializa a tabela Q
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.random.uniform(low=-0.01, high=0.01, size=(env.observation_space.n, num_actions))

    # guarda a recompensa de cada passo
    rewards_per_step = []

    state, _ = env.reset()
    reward = 0

    # loop principal
    for i in range(total_steps):

        # escolhe a próxima ação -- usa epsilon-greedy
        action = epsilon_greedy(Q, state, epsilon)

        # realiza a ação, ou seja, dá um passo no ambiente
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        if terminated:
            # para estados terminais
            V_next_state = 0
        else:
            # para estados não-terminais -- valor máximo (melhor ação)
            V_next_state = np.max(Q[next_state,:])

        # atualiza a Q-table
        # delta = (estimativa usando a nova recompensa) - estimativa antiga
        delta = (reward + gamma * V_next_state) - Q[state,action]
        Q[state,action] = Q[state,action] + lr * delta

        rewards_per_step.append(reward)

        if done:
            state, _ = env.reset()
        else:
            state = next_state

        # a cada 1000 passos, imprime informação sobre o progresso
        if verbose and ((i+1) % 1000 == 0):
            avg_reward = np.mean(rewards_per_step[-100:])
            print(f"Step {i+1} Average Reward (last 100): {avg_reward:.3f}")

    return rewards_per_step, Q

In [ ]:
TOTAL_STEPS = 1_000
LR = 0.2
GAMMA = 0.70   # só deve dar a política ótima para valores a partir de 0.85
EPSILON = 0.1

rmax = TOTAL_STEPS//2
env = TwoChoiceEnv()

rewards1, qtable1 = run_qlearning_step(env, TOTAL_STEPS, LR, GAMMA, EPSILON, verbose=True)
print("Acumulado final =", sum(rewards1))

In [ ]:
# Mostra um gráfico de passos x retornos não descontados acumulados
plot_result(rewards1, rmax, cumulative='avg')

Vamos conferir o valor (retorno esperado) de cada ação no estado 0:

In [ ]:
qtable1[0]

## 3 - Differential Q-Learning

Um algoritmo específico para tarefas continuadas.

Ele é baseado na **formulação de recompensa média** para as tarefas (e os MDPs). Nessa formulação:
- é usado um *retorno* baseado nas diferenças entre cada recompensas real $R_t$ e a recompensa média até o passo $t$
- não existe o fator de desconto $\gamma$ (gamma)
- mas há um novo parâmetro $\eta$ (êta) que controla a "taxa de aprendizagem" da recompensa média estimada

In [7]:
# Algoritmo Differential Q-learning
def run_differential_qlearning_step(env, total_steps, lr=0.1, lr_mean=0.1, epsilon=0.1, verbose=False):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n

    # inicializa a tabela Q
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.random.uniform(low=-0.01, high=0.01, size=(env.observation_space.n, num_actions))

    # guarda a recompensa de cada passo
    rewards_per_step = []
    states = []

    state, _ = env.reset()
    reward = 0
    mean_reward = 0.0

    # loop principal
    for i in range(total_steps):

        # escolhe a próxima ação -- usa epsilon-greedy
        action = epsilon_greedy(Q, state, epsilon)

        # realiza a ação, ou seja, dá um passo no ambiente
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        assert not done, "This algorithm is for infinite tasks!"

        # valor do próximo estado - valor máximo (melhor ação)
        V_next_state = np.max(Q[next_state,:])

        # atualiza a Q-table
        # delta = (estimativa usando a nova recompensa) - estimativa antiga
        delta = (reward - mean_reward + V_next_state) - Q[state,action]
        Q[state,action] = Q[state,action] + lr * delta

        # atualiza a recompensa média
        mean_reward += lr_mean * delta
        # alt.: mean_reward += lr_mean * (reward - mean_reward) # maior variância

        states.append(state)
        rewards_per_step.append(reward)
        state = next_state

        # a cada 1000 passos, imprime informação sobre o progresso
        if verbose and ((i+1) % 1000 == 0):
            avg_reward = np.mean(rewards_per_step[-100:])
            print(f"Step {i+1} Average Reward (last 100): {avg_reward:.3f}")

    return rewards_per_step, Q

In [ ]:
TOTAL_STEPS = 10_000
LR      = 0.4
LR_MEAN = 0.01
EPSILON = 0.1

rmax = TOTAL_STEPS//2
env = TwoChoiceEnv(coherent_action=False)

rewards2, qtable2 = run_differential_qlearning_step(env, TOTAL_STEPS, LR, LR_MEAN, EPSILON, verbose=True)
print("Acumulado final =", sum(rewards2))

assert np.isnan(qtable2).sum() == 0

Valores das ações no estado **0**:

In [ ]:
qtable2[0]

In [ ]:
# Mostra um gráfico de passos x retornos não descontados acumulados
plot_result(rewards2, rmax, cumulative='avg')

## 4 - Otimizando Parâmetros

Vamos usar a biblioteca *Optuna* para otimizar os (hiper-)parâmetros dos algoritmos de treinamento.

In [11]:
ENV = TwoChoiceEnv()
#environment = FromDiscreteTupleToDiscreteObs( AccessControlEnv() )

RUNS_PER_TRIAL = 10
TRAIN_STEPS = 500

### 4.1 - Q-Learning

Vamos rodar duas otimizações separadas para estes dois valores de `gamma`: `0.7` e `0.9`.

Espera-se que o segundo valor (`0.9`) permita o Q-Learning chegar a um resultado ótimo, mas o primeiro valor (`0.7`) não.

In [12]:
def create_train_fn(fixed_gamma):
    def train(trial : optuna.Trial):
        # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
        lr = trial.suggest_float('lr', 0.01, 1.0)
        eps = trial.suggest_float('epsilon', 0.01, 1.00)

        print(f"\nTRIAL #{trial.number}: lr={lr}, eps={eps}")

        # roda o algoritmo várias vezes
        results = repeated_exec(RUNS_PER_TRIAL, "qlearn-optuna", run_qlearning_step, ENV, TRAIN_STEPS, lr=lr, epsilon=eps, gamma=fixed_gamma)

        # média das somas das recompensas de cada treinamento
        # results[1] -> guarda as listas de recompensas dos vários treinamentos, com shape (treinamentos x passos)
        return results[1].sum(axis=1).mean()
    return train

In [ ]:
GAMMA = 0.70
study = optuna.create_study(direction='maximize',
                        storage='sqlite:///optuna_continuing.db',
                        study_name=f'qlearning-g{GAMMA}',
                        load_if_exists=True)

study.optimize(create_train_fn(GAMMA), n_trials=50)
clear_output()

print("MELHORES PARÂMETROS PARA GAMMA", GAMMA, ":")
print(study.best_params)
qlearn_params_g07 = study.best_params
qlearn_params_g07['gamma'] = GAMMA

In [ ]:
GAMMA = 0.90
study = optuna.create_study(direction='maximize',
                        storage='sqlite:///optuna_continuing.db',
                        study_name=f'qlearning-g{GAMMA}',
                        load_if_exists=True)

study.optimize(create_train_fn(GAMMA), n_trials=50)
clear_output()

print("MELHORES PARÂMETROS PARA GAMMA", GAMMA, ":")
print(study.best_params)
qlearn_params_g09 = study.best_params
qlearn_params_g09['gamma'] = GAMMA

### 4.2 - Differential Q-Learning

In [15]:
def train_diff(trial : optuna.Trial):
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    lr = trial.suggest_float('lr', 0.01, 1.0)
    eps = trial.suggest_float('epsilon', 0.01, 1.00)
    lrmean = trial.suggest_float('lr_mean', 0.01, 1.0)

    print(f"\nTRIAL #{trial.number}: {lr=}, {eps=}, {lrmean=}")

    # roda o algoritmo várias vezes
    results = repeated_exec(RUNS_PER_TRIAL, "diff-qlearn-optuna", run_differential_qlearning_step, ENV, TRAIN_STEPS, lr=lr, epsilon=eps, lr_mean=lrmean)

    # média das somas das recompensas de cada treinamento
    return results[1].sum(axis=1).mean()


In [ ]:
study = optuna.create_study(direction='maximize',
                        storage='sqlite:///optuna_continuing.db',
                        study_name='diff-qlearning',
                        load_if_exists=True)

study.optimize(train_diff, n_trials=100)
clear_output()

print("MELHORES PARÂMETROS:")
print(study.best_params)
diff_qlearn_params = study.best_params

## 5 - Experimentos

### 5.1 - Desempenho no Treinamento

Comparando os dois usando os parâmetros ótimos obtidos antes.

In [17]:
TRAIN_STEPS = 1_000
RUNS = 200

results1 = []

results1.append( repeated_exec(RUNS, f"Diff Q-Learning", run_differential_qlearning_step, ENV, TRAIN_STEPS, **diff_qlearn_params) )
clear_output()

results1.append( repeated_exec(RUNS, f"Q-Learning (g=0.7)", run_qlearning_step, ENV, TRAIN_STEPS, **qlearn_params_g07) )
clear_output()

results1.append( repeated_exec(RUNS, f"Q-Learning (g=0.9)", run_qlearning_step, ENV, TRAIN_STEPS, **qlearn_params_g09) )
clear_output()

In [ ]:
plot_multiple_results(results1, cumulative='sum', window=1)

In [ ]:
plot_multiple_results(results1, cumulative='sum', window=1, plot_stddev=True)

In [20]:
#index = 0
#plot_multiple_results(results1[index:index+1], cumulative='sum', window=1, plot_stddev=True)

### 5.2 - Desempenho Pós-Treinamento

Os experimentos abaixo avaliam o agente após o treinamento. Para isso, o loop treinamento-avaliação é repetido várias vezes.

Surpreendentemente, os resultados variam muito. Mas, espera-se que, no ambiente `TwoChoiceEnv`: 
- o *Differential Q-Learning* e o *Q-Learning* com gammas altos (como `0.9`) atinjam o ótimo, 
- mas o *Q-Learning* com valor de gamma baixo (como `0.7`) não atinja o ótimo.

In [21]:
from util.qtable_helper import evaluate_qtable_policy

In [22]:
TRAIN_STEPS = 1_000
TEST_STEPS = 100
REPETITIONS = 300

results_dict = { "Diff-QLearn-Greedy": np.zeros(shape=(REPETITIONS,)),
                 "QLearn(0.7)-Greedy": np.zeros(shape=(REPETITIONS,)),
                 "QLearn(0.9)-Greedy": np.zeros(shape=(REPETITIONS,)) }

for i in range(REPETITIONS):
    print(f"({i}) Treinando o Differential Q-Learning")
    _, qtable1 = run_differential_qlearning_step(ENV, TRAIN_STEPS, **diff_qlearn_params, verbose=False)

    print(f"({i}) Treinando o Q-Learning (g=0.7)")
    _, qtable2 = run_qlearning_step(ENV, TRAIN_STEPS, **qlearn_params_g07, verbose=False)

    print(f"({i}) Treinando o Q-Learning (g=0.9)")
    _, qtable3 = run_qlearning_step(ENV, TRAIN_STEPS, **qlearn_params_g09, verbose=False)

    print("------ ")
    print(f"({i}) Executando políticas 'greedy' com as Q-tables treinadas:")
    
    mean_reward, _ = evaluate_qtable_policy(ENV, qtable1, num_episodes=1)
    results_dict["Diff-QLearn-Greedy"][i] = mean_reward
    
    mean_reward, _ = evaluate_qtable_policy(ENV, qtable2, num_episodes=1)
    results_dict["QLearn(0.7)-Greedy"][i] = mean_reward
    
    mean_reward, _ = evaluate_qtable_policy(ENV, qtable3, num_episodes=1)
    results_dict["QLearn(0.9)-Greedy"][i] = mean_reward

    clear_output()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract keys and calculate means and stds
algorithms = results_dict.keys()
means = [np.mean(results_dict[x]) for x in algorithms]
stds = [np.std(results_dict[x]) for x in algorithms]

# Create bar plot with error bars
plt.figure(figsize=(10, 6))
plt.bar(algorithms, means, yerr=stds, capsize=5)

# Customize the plot
plt.xticks(rotation=45, ha='right')
plt.ylabel('Recompensa (Média)')
plt.title('Desempenho Médio Após Treinamentos, com Desvio Padrão')

# Adjust layout to prevent label cutoff
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
means, stds

## 6 - Desafio

Cada um dos algoritmos básicos de TD-Learning da formulação padrão (com retorno descontado) tem um correspondente na formulação de recompensa média.

O **Differential Q-Learning**, que vimos aqui, é o correspondente do *Q-Learning*.

O desafio é este:
1. Implementar um **Differential SARSA**
1. Otimizar seus parâmetros
1. Rodar experimentos comparando-o com os algoritmos anteriores

Também destacamos que, no livro de Sutton & Barto tem uma implementação do **Differential SARSA de n passos** usando *aproximação de função*.
